In [42]:
from typing import List, Tuple
from datetime import datetime, timedelta
import numpy.typing as npt
import numpy as np
import os
import cv2
import pickle
from tqdm import tqdm
import itertools

from utils.data_dirs import data_dirs

In [43]:
raw_dir, processed_dir, folder, EXPERIMENT_DATA, suffix = data_dirs(False)
print(raw_dir)
print(processed_dir)
print(folder)
print(EXPERIMENT_DATA)
print(suffix)

/Users/chanwutk/Documents/apperception/data/v1.0-mini
/Users/chanwutk/Documents/apperception/data/v1.0-mini
v1.0-mini
/work/apperception/data/raw/scenic/experiment_data



In [44]:
fps = 20

In [45]:
interpolate = False

In [46]:
partition_dir = os.path.join(processed_dir, 'partition')

In [47]:
cameras = [
    'CAM_FRONT',
    'CAM_FRONT_LEFT',
    'CAM_FRONT_RIGHT',
    'CAM_BACK_LEFT',
    'CAM_BACK_RIGHT',
    'CAM_BACK',
]

In [48]:
def get_videos(sample_data):
    videos = {}
    for t in sample_data.itertuples(index=False):
        scene = t.scene_name
        file = t.filename.split('/')[1]
        key = f"{scene}-{file}"
        if key not in videos:
            videos[key] = []
        t = (
            t.scene_name,
            t.token,
            t.frame_order,
            t.filename,
            t.camera_translation,
            t.camera_rotation,
            t.camera_intrinsic,
            t.ego_translation,
            t.ego_rotation,
            datetime.fromtimestamp(t.timestamp / 1_000_000),
            t.camera_heading,
            t.ego_heading
        )
        videos[key].append(t)

    for video in videos.values():
        video.sort(key=lambda v: v[9])
            
    return videos

In [49]:
rame = tuple

def frame_with_best_alignments(scene: "List[Frame]") -> "Frame":
    max_aligned_frames = -1
    frame_idx = None
    frame_t = None
    for idx, frame in enumerate(scene):
        t = frame[9]
        aligned_frames = 0
        for _frame in scene:
            _t = _frame[9]
            sec_diff = (t - _t).total_seconds()
            if (sec_diff * fps).is_integer():
                aligned_frames += 1
        if aligned_frames > max_aligned_frames:
            max_aligned_frames = aligned_frames
            frame_idx = idx
            frame_t = t
    
    return frame_idx, frame_t


def load_image(cache: "Dict[str, npt.NDArray]", name: str) -> "npt.NDArray":
    if name not in cache:
        cache[name] = cv2.imread(os.path.join(raw_dir, name))
    return cache[name]
    

def compose(location: str, name: str, scene: "List[Frame]"):
    t_start = scene[0][9]
    t_end = scene[-1][9]

    frame_idx, frame_t = frame_with_best_alignments(scene)
    
    frames_before = int((frame_t - t_start).total_seconds() * fps)
    
    def get_timestamp(idx: int):
        return frame_t - timedelta(seconds=(frames_before - idx) / fps)
    
    assert get_timestamp(0) >= t_start, f'{get_timestamp(0)} {t_start}'
    
    frames: "List[npt.NDArray]" = []
    frame_idx = 0
    img_cache = {}
    print("Reading scene:", name)
    n = 0
    while get_timestamp(n) < t_end:
        t = get_timestamp(n)
        n += 1

    for i in tqdm(itertools.count(), total=n):
        if get_timestamp(i) >= t_end:
            break
#         print(i)
        t = get_timestamp(i)
        while scene[frame_idx + 1][9] < t:
            frame_idx += 1
        f_curr, t_curr = scene[frame_idx][3::6]
        f_next, t_next = scene[frame_idx + 1][3::6]
        
        img_curr = load_image(img_cache, f_curr)
        img_next = load_image(img_cache, f_next)
#         img_curr = cv2.imread(os.path.join(BASE_DIR, f_curr))
#         img_next = cv2.imread(os.path.join(BASE_DIR, f_next))
        
        ratio = (t - t_curr).total_seconds() / (t_next - t_curr).total_seconds()
        
        if interpolate:
            img = (img_curr * (1 - ratio) + img_next * ratio).astype(np.uint8)
        else:
            if ratio < 0.5:
                img = img_curr
            else:
                img = img_next
        frames.append(img)
    
    t0 = get_timestamp(0)
    t0_tuple = (t0.year, t0.month, t0.day, t0.hour, t0.minute, t0.second, t0.microsecond)
    filename = f'{name}-{"-".join([*map(str, t0_tuple)])}.mp4'
    base = os.path.join(processed_dir, 'videos', location)
    if not os.path.exists(base):
        os.makedirs(base)
    out = cv2.VideoWriter(
        os.path.join(base, filename),
        cv2.VideoWriter_fourcc(*'mp4v'),
        fps,
        (1600, 900)
    )
    print(f"Writing scene ({os.path.join(base, filename)}):")
    for frame in tqdm(frames):
        out.write(frame)
    out.release()
    cv2.destroyAllWindows()
    
    return frames, filename, get_timestamp(0)

In [50]:
for l in os.listdir(partition_dir):
    d = os.path.join(partition_dir, l)
    with open(os.path.join(d, 'sample_data.pickle'), "rb") as f:
        sample_data = pickle.load(f)
    videos = get_videos(sample_data)
    
    
    output = {}
    for name in videos:
        video = videos[name]
        frames, filename, start = compose(l, name, video)
        output[name] = {
            "filename": filename,
            "start": start,
            "columns": [
                "cameraId",
                "frameId",
                "frameNum",
                "filename",
                "cameraTranslation",
                "cameraRotation",
                "cameraIntrinsic",
                "egoTranslation",
                "egoRotation",
                "timestamp",
                "cameraHeading",
                "egoHeading",
            ],
            "frames": video
        }
        print("done", filename)
    base = os.path.join(processed_dir, 'videos', l)
    with open(os.path.join(base, 'frames.pickle'), "wb") as f:
        pickle.dump(output, f)

Reading scene: scene-0103-CAM_FRONT_LEFT


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 388/388 [00:04<00:00, 94.94it/s]


Writing scene (/Users/chanwutk/Documents/apperception/data/v1.0-mini/videos/boston-seaport/scene-0103-CAM_FRONT_LEFT-2018-8-1-12-26-43-504799.mp4):


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 388/388 [00:04<00:00, 84.11it/s]


done scene-0103-CAM_FRONT_LEFT-2018-8-1-12-26-43-504799.mp4
Reading scene: scene-0103-CAM_FRONT


...

done scene-1100-CAM_BACK-2018-11-21-3-49-47-937525.mp4
Reading scene: scene-1100-CAM_BACK_LEFT


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 390/390 [00:03<00:00, 97.79it/s]


Writing scene (/Users/chanwutk/Documents/apperception/data/v1.0-mini/videos/singapore-hollandvillage/scene-1100-CAM_BACK_LEFT-2018-11-21-3-49-47-947423.mp4):


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 390/390 [00:05<00:00, 65.61it/s]


done scene-1100-CAM_BACK_LEFT-2018-11-21-3-49-47-947423.mp4
